In [ ]:
!sudo apt install tesseract-ocr
!sudo apt-get install poppler-utils
!pip install pdf2image
!pip install pytesseract
!pip install neo4j
!pip install cohere tiktoken
!sudo apt-get update
!sudo apt-get install poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,729 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

## Reading a PDF document with OCR

In [ ]:
import requests
import pdf2image
import pytesseract

pdf = requests.get('https://arxiv.org/pdf/2110.03526.pdf')
doc = pdf2image.convert_from_bytes(pdf.content)
#print(doc)
# Get the article text
article = []
for page_number, page_data in enumerate(doc):
    txt = pytesseract.image_to_string(page_data).encode("utf-8")
    # Sixth page are only references
    if page_number < 6:
      article.append(txt.decode("utf-8"))
article_txt = " ".join(article)

## Biomedical Name Entity Linking

In [ ]:
import nltk
nltk.download('punkt')

def clean_text(text):
  """Remove section titles and figure descriptions from text"""
  clean = "\n".join([row for row in text.split("\n") if (len(row.split(" "))) > 3 and not (row.startswith("(a)"))
                    and not row.startswith("Figure")])
  return clean

ctext = clean_text(article_txt)
sentences = nltk.tokenize.sent_tokenize(ctext)
print(sentences[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Tissue Engineering and Regeneration of Skin
and Hair Follicle Growth From Stem Cells
Many people with skin diseases such as chronic wounds, non-healing and diabetic
ulcers need reconstruction and regeneration of their skin.


In [ ]:
import requests

def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

entity_list = []
for s in sentences[:-1]:
    #print(query_plain(s))
    entity_list.append(query_plain(s))



In [ ]:
print(entity_list)

In [ ]:
import hashlib

In [ ]:
parsed_entities = []
for entities in entity_list:
  e = []
  # If there are not entities in the text
  if not entities.get('annotations'):
    parsed_entities.append({'text':entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})
    continue
  for entity in entities['annotations']:
    other_ids = [id for id in entity['id'] if not id.startswith("BERN2")]
    entity_type = entity['obj']
    entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
    try:
      entity_id = [id for id in entity['id'] if id.startswith("BERN2")][0]
    except IndexError:
      entity_id = entity_name
    e.append({'entity_id': entity_id, 'other_ids': other_ids, 'entity_type': entity_type, 'entity': entity_name})
  parsed_entities.append({'entities':e, 'text':entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})

In [ ]:
"""parsed_entities = []

for entities in entity_list:
    e = []
    # If there are no entities in the text
    if not entities.get('annotations'):
        parsed_entities.append({'text': entities['text'], 'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()})
        continue

    for entity in entities['annotations']:
        other_ids = [id for id in entity['id'] if not id.startswith("BERN2")]
        entity_type = entity['obj']
        entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
        try:
            entity_id = [id for id in entity['id'] if id.startswith("BERN2")][0]
        except IndexError:
            entity_id = entity_name

        e.append({'entity_id': entity_id, 'other_ids': other_ids, 'entity_type': entity_type, 'entity': entity_name})

    parsed_entities.append({
        'entities': e,
        'text': entities['text'],
        'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()
    })
"""
# Print the first entry in parsed_entities



'parsed_entities = []\n\nfor entities in entity_list:\n    e = []\n    # If there are no entities in the text\n    if not entities.get(\'annotations\'):\n        parsed_entities.append({\'text\': entities[\'text\'], \'text_sha256\': hashlib.sha256(entities[\'text\'].encode(\'utf-8\')).hexdigest()})\n        continue\n\n    for entity in entities[\'annotations\']:\n        other_ids = [id for id in entity[\'id\'] if not id.startswith("BERN2")]\n        entity_type = entity[\'obj\']\n        entity_name = entities[\'text\'][entity[\'span\'][\'begin\']:entity[\'span\'][\'end\']]\n        try:\n            entity_id = [id for id in entity[\'id\'] if id.startswith("BERN2")][0]\n        except IndexError:\n            entity_id = entity_name\n\n        e.append({\'entity_id\': entity_id, \'other_ids\': other_ids, \'entity_type\': entity_type, \'entity\': entity_name})\n\n    parsed_entities.append({\n        \'entities\': e,\n        \'text\': entities[\'text\'],\n        \'text_sha256\': ha

In [ ]:
print(parsed_entities[0])

{'text': 'Good morning miss, thank you for coming to our clinic, I would like to confirm the information that was gotten from the nurse.', 'text_sha256': '3e55d5441dc7572b81f48a4710cc2ba8efd29846d09a7736a33ef9727fc80f1c'}


In [ ]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://3.86.43.95:7687'
user = 'neo4j'
password = 'interiors-reduction-reasons'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
       # session.commit()
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

## Entity Extraction Logic

In [ ]:
doctor = article_txt.split("\n")[0]
patient = " ".join(article_txt.split("\n")[2:4])

print(doctor)
print(patient)
neo4j_query("""
MERGE (a:Doctor{name:$doctor})
MERGE (b:Article{patient:$patient})
MERGE (a)-[:WROTE]->(b)
""", {'patient':patient, 'doctor':doctor})

Mohammadreza Ahmadi
Tissue Engineering and Regeneration of Skin and Hair Follicle Growth From Stem Cells


""


In [ ]:
neo4j_query("""
MATCH (a:Article)
UNWIND $data as row
MERGE (s:Sentence{id:row.text_sha256})
SET s.text = row.text
MERGE (a)-[:HAS_SENTENCE]->(s)
WITH s, row.entities as entities
UNWIND entities as entity
MERGE (e:Entity{id:entity.entity_id})
ON CREATE SET e.other_ids = entity.other_ids,
              e.name = entity.entity,
              e.type = entity.entity_type
MERGE (s)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count + 1
""", {'data': parsed_entities})

""


In [ ]:
neo4j_query("""
MATCH (e:Entity)<-[:MENTIONS]-(s:Sentence)
WHERE e.name = "autoimmune diseases"
RETURN s.text as result
""")

,result
0,"These cells, later found to be hematopoietic s..."


In [ ]:
neo4j_query("""
MATCH (e1:Entity)<-[:MENTIONS]-()-[:MENTIONS]->(e2:Entity)
WHERE id(e1) < id(e2)
RETURN e1.name as entity1, e2.name as entity2, count(*) as cooccurrence
ORDER BY cooccurrence
DESC LIMIT 3
""")

,entity1,entity2,cooccurrence
0,collagen,fibroblasts,6
1,mesenchymal stem cells,collagen,4
2,mesenchymal stem cells,fibroblasts,4


In [ ]:
neo4j_query("""
MATCH (a:Author)-[:WROTE]->()-[:HAS_SENTENCE]->()-[:MENTIONS]->(e:Entity)
RETURN a.name as author, e.name as entity, count(*) as count
ORDER BY count DESC
LIMIT 5
""")

,author,entity,count


In [ ]:
!pip install --upgrade pip setuptools



In [ ]:

!pip install -U numpy==1.19.1 --no-cache-dir

In [ ]:
!apt-get install -y libblas-dev liblapack-dev



In [ ]:
!pip install zero-shot-re


In [ ]:
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor

model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
relations = ['associated', 'interacts']
extractor = RelationExtractor(model, tokenizer, relations)